In [1]:
# This is a solution for the Keggle Titanic challenge
# by Th. Kätzel, 09/2016

import pandas as pd
import numpy as np

df = pd.read_csv('train.csv', header = 0)

In [2]:
df.dtypes
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
df['Age'].median()
#df[['Age','Pclass','Fare']]
#df[df['Age'] > 60][['Sex', 'Age', 'Pclass', 'Survived']]

# remove NaN-Age from dataset
df[df['Age'].isnull()][['Sex', 'Age', 'Pclass', 'Survived']].head()

,Sex,Age,Pclass,Survived
5,male,NaN,3,0
17,male,NaN,2,1
19,female,NaN,3,1
26,male,NaN,3,0
28,female,NaN,3,1


In [4]:
for i in range(1,4):
    print i, len(df[(df['Sex'] == 'male') & (df['Pclass'] == i)])

1 122
2 108
3 347


In [5]:
import pylab as pl
df['Age'].dropna().hist(bins=40)
pl.show()

In [6]:
df['Gender'] = df['Sex'].map({'female' : 1, 'male' : 0})#.astype(int)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,0


In [7]:
df['AgeFill'] = df['Age']
# engineer feature AgeFill (with median values where NULL)
df.loc[(df['Age'].isnull()),'AgeFill'] = df['Age'].median()

df['AgeIsNull'] = pd.isnull(df.Age).astype(int)

# engineer feature family size (with Parents/children on board + siblings on board)
df['FamSize'] = df['Parch'] + df['SibSp']

df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender,AgeFill,AgeIsNull,FamSize
886,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.00,NaN,S,0,27,0,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S,1,19,0,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,1,28,1,3
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C,0,26,0,0
890,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,NaN,Q,0,32,0,0


In [8]:
#df['Age*Class'] = df.AgeFill * df.Pclass
#df['Age*Class'].hist()
#pl.show()

# remove all string data types from the training set
df.dtypes
df = df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Fare'], axis=1)

In [9]:
df = df.drop(['Age'], axis=1)
trainset = df.values
df.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Gender,AgeFill,AgeIsNull,FamSize
0,1,0,3,1,0,0,22,0,1
1,2,1,1,1,0,1,38,0,1
2,3,1,3,0,0,1,26,0,0
3,4,1,1,1,0,1,35,0,1
4,5,0,3,0,0,0,35,0,0


In [10]:
trainset[0::,2::]

array([[  3.,   1.,   0., ...,  22.,   0.,   1.],
       [  1.,   1.,   0., ...,  38.,   0.,   1.],
       [  3.,   0.,   0., ...,  26.,   0.,   0.],
       ..., 
       [  3.,   1.,   2., ...,  28.,   1.,   3.],
       [  1.,   0.,   0., ...,  26.,   0.,   0.],
       [  3.,   0.,   0., ...,  32.,   0.,   0.]])

In [11]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 200)

forest = forest.fit(trainset[0::,2::],trainset[0::,1])

In [12]:
dft = pd.read_csv('test.csv', header = 0)
#do Gender
dft['Gender'] = dft['Sex'].map({'female' : 1, 'male' : 0}).astype(int)
#do AgeFill
dft['AgeFill'] = dft['Age']
# engineer feature AgeFill (with median values where NULL)
dft.loc[(dft['Age'].isnull()),'AgeFill'] = dft['Age'].median() # would df['Age'] be better?
#do AgeIsNull
dft['AgeIsNull'] = pd.isnull(dft.Age).astype(int)
#do FamSize
dft['FamSize'] = dft['Parch'] + dft['SibSp']
dft = dft.drop(['Name', 'Sex', 'Age', 'Ticket', 'Cabin', 'Embarked', 'Fare'], axis=1)
testset = dft.values
dft.head()

,PassengerId,Pclass,SibSp,Parch,Gender,AgeFill,AgeIsNull,FamSize
0,892,3,0,0,0,34.5,0,0
1,893,3,1,0,1,47.0,0,1
2,894,2,0,0,0,62.0,0,0
3,895,3,0,0,0,27.0,0,0
4,896,3,1,1,1,22.0,0,2


In [13]:
dft.dtypes
testset[0::,1::]

array([[  3. ,   0. ,   0. , ...,  34.5,   0. ,   0. ],
       [  3. ,   1. ,   0. , ...,  47. ,   0. ,   1. ],
       [  2. ,   0. ,   0. , ...,  62. ,   0. ,   0. ],
       ..., 
       [  3. ,   0. ,   0. , ...,  38.5,   0. ,   0. ],
       [  3. ,   0. ,   0. , ...,  27. ,   1. ,   0. ],
       [  3. ,   1. ,   1. , ...,  27. ,   1. ,   2. ]])

In [57]:
output = forest.predict(testset[0::,1::])
#submit = testset['PassengerID']+output
#type(output)
#len(output)
#type(testset[0::,0])
#len(testset[0::,0])
connum = np.vstack((testset[0::,0],output)).T
#submit2 = px.reshape((-1,3))
sub = pd.DataFrame({'PassengerId':connum[:,0],'Survived':connum[:,1]})
sub2 = sub.astype(int)
sub2.dtypes
sub2.to_csv('sub1.csv', index=False)